In [15]:
import numpy as np 
import pandas as pd 
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pickle
from numba import jit, cuda

In [16]:
train = pd.read_csv("./data/train.csv")
stocks = train['stock_id'].unique()
train = train.sort_values(['date_id','time_id'])
train.head()



,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [17]:
def split(data,var='wap',testing_size=100,pred_size=10):
    X = data.iloc[0:data.shape[0]-pred_size]
    y = data[var].iloc[pred_size:]

    X_train = X.iloc[:X.shape[0]-testing_size]
    X_test = X.iloc[X.shape[0]-testing_size:]

    y_train = y.iloc[:y.shape[0]-testing_size]
    y_test = y.iloc[y.shape[0]-testing_size:].reset_index(drop=True)

    trainTest = {
        'X_train':X_train,
        'X_test':X_test,
        'y_train':y_train,
        'y_test':y_test
    }
    return trainTest

In [18]:
@jit(target_backend='cuda')
def createForecasters(data,var='wap'):
    model_fits = {}
    X_train = data['X_train']
    for stock in stocks:
        print("stock: ",stock)
       
        model = ARIMA(X_train[X_train['stock_id'] == stock][var], order=(2,0,0))
        model_fits[stock] = model.fit()
        
    return model_fits
        
        

C:\Users\adams\AppData\Local\Temp\ipykernel_5552\721257731.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


In [19]:
@jit(target_backend='cuda')
def createModel(data,forecasters,var='wap',forecast_dist = 6,method='mean'):
    results = {}
    X_test = data['X_test']
    y_hat = {}
    for stock in stocks:
        print("forcasting stock:",stock)
        y_hat[stock] = []
        i = 0
        for x in X_test[X_test['stock_id'] == stock][var]:
            forecast = forecasters[stock].forecast(forecast_dist)

            if method == 'mean':
                y_hat[stock].append(np.mean(forecast))
            elif method == 'last':
                y_hat[stock].append(forecast.iloc[-1])
            else:
                print("invalid method")
                break
            
            if i % 100 == 0:
                print('run:',i)
            forecasters[stock] = forecasters[stock].append([x], refit=False)
            
            i += 1




    y_hat_df = pd.DataFrame(y_hat)
    
    for stock in stocks:
        print("creating modl for stock:",stock)
        target = X_test[X_test['stock_id'] == stock]
        target.reset_index(inplace=True)


        ols_model = sm.OLS(list(target['target']),y_hat_df)
        results[stock] = ols_model.fit()
        print(results[stock].summary())

    return results

C:\Users\adams\AppData\Local\Temp\ipykernel_5552\1928463893.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(target_backend='cuda')


In [20]:
trainTest = split(train,var='wap',testing_size=500000)


In [21]:
forecasters = createForecasters(trainTest)
with open('forecasters.pickle', 'wb') as handle:
        pickle.dump(forecasters, handle, protocol=pickle.HIGHEST_PROTOCOL)

C:\Users\adams\AppData\Local\Temp\ipykernel_5552\721257731.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "createForecasters" failed type inference due to: Untyped global name 'ARIMA': Cannot determine Numba type of <class 'type'>

File "..\..\..\AppData\Local\Temp\ipykernel_5552\721257731.py", line 8:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\adams\AppData\Local\Temp\ipykernel_5552\721257731.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "createForecasters" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "..\..\..\AppData\Local\Temp\ipykernel_5552\721257731.py", line 5:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function "createForeca

stock:  0
stock:  1


c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


stock:  2


c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


...

c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


stock:  135


c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


stock:  102


c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [22]:
models = createModel(trainTest,forecasters,method='last')
with open('models.pickle', 'wb') as handle:
        pickle.dump(models, handle, protocol=pickle.HIGHEST_PROTOCOL)

C:\Users\adams\AppData\Local\Temp\ipykernel_5552\1928463893.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "createModel" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\adams\AppData\Local\Temp\ipykernel_5552\1928463893.py (1)

File "..\..\..\AppData\Local\Temp\ipykernel_5552\1928463893.py", line 1:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
C:\Users\adams\AppData\Local\Temp\ipykernel_5552\1928463893.py:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "createModel" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "..\..\..\AppData\Local\Temp\ipykernel_5552\1928463893.py", line 6:
<source missing, REPL/exec in use?>

  @jit(target_backend='cuda')
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\numba\core\object_mode_passes.py:1

forcasting stock: 0


c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an inte

run: 0
run: 100
run: 200
run: 300
run: 400
run: 500
run: 600
run: 700
run: 800
run: 900
run: 1000
run: 1100
run: 1200
run: 1300
run: 1400
run: 1500
run: 1600
run: 1700
run: 1800
run: 1900
run: 2000
run: 2100
run: 2200
run: 2300
run: 2400
forcasting stock: 1
run: 0


c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an inte

...

run: 100
run: 200
run: 300
run: 400
run: 500
run: 600
run: 700
run: 800
run: 900
run: 1000
run: 1100
run: 1200
run: 1300
run: 1400
run: 1500
run: 1600
run: 1700
run: 1800
run: 1900
run: 2000
run: 2100
run: 2200
run: 2300
run: 2400
forcasting stock: 160


c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


run: 0


c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\adams\Documents\projects\comp\.conda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


run: 100
run: 200
run: 300
run: 400
run: 500


In [ ]:
with open('models.pickle', 'wb') as handle:
        pickle.dump(models, handle, protocol=pickle.HIGHEST_PROTOCOL)